In [70]:
from datasets import load_dataset

data = load_dataset(
    "jamescalam/llama-2-arxiv-papers-chunked",
    split="train"
)
data

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 4838
})

In [71]:
data[0]

{'doi': '1102.0183',
 'chunk-id': '0',
 'chunk': 'High-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nTechnical Report No. IDSIA-01-11\nJanuary 2011\nIDSIA / USI-SUPSI\nDalle Molle Institute for Arti\x0ccial Intelligence\nGalleria 2, 6928 Manno, Switzerland\nIDSIA is a joint institute of both University of Lugano (USI) and University of Applied Sciences of Southern Switzerland (SUPSI),\nand was founded in 1988 by the Dalle Molle Foundation which promoted quality of life.\nThis work was partially supported by the Swiss Commission for Technology and Innovation (CTI), Project n. 9688.1 IFF:\nIntelligent Fill in Form.arXiv:1102.0183v1  [cs.AI]  1 Feb 2011\nTechnical Report No. IDSIA-01-11 1\nHigh-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nJanuary 2011\nAbs

In [72]:
data = data.map(lambda x: {
    'uid': f"{x['doi']}-{x['chunk-id']}"
})
data

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references', 'uid'],
    num_rows: 4838
})

In [73]:
data = data.to_pandas()
# drop irrelevant fields
data = data[['uid', 'chunk', 'title', 'source']]

In [74]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [75]:
from openai import OpenAI

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
embed_model_id = "text-embedding-ada-002"
res = client.embeddings.create(
    input=[
        "We would have some text to embed here",
        "And maybe another chunk here too"
    ],model=embed_model_id)

In [76]:
len(res.data[0].embedding), len(res.data[1].embedding)

(1536, 1536)

In [77]:
from pinecone import Pinecone, PodSpec
# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get("PINECONE_API_KEY")
# find your environment next to the api key in pinecone console
env = os.environ.get("PINECONE_ENVIRONMENT")

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))


In [78]:
pc.create_index(
  name="nemo-guardrails-rag-with-actions",
  dimension=1536,
  metric="cosine",
  spec=PodSpec(
    environment="gcp-starter",
    pod_type="p1.x1",
    pods=1
  )
)

ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': 'f0b1f9ab2fe7ec055fc0334cca02ad6c', 'Date': 'Thu, 04 Apr 2024 08:44:55 GMT', 'Server': 'Google Frontend', 'Content-Length': '101', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"FORBIDDEN","message":"Projects only support a single Starter index."},"status":403}


In [79]:
index = pc.Index("nemo-guardrails-rag-with-actions")

In [84]:
from tqdm.auto import tqdm

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(data), batch_size)):
    # find end of batch
    i_end = min(len(data), i+batch_size)
    batch = data[i:i_end]
    # get ids
    ids_batch = batch['uid'].to_list()
    # get texts to encode
    texts = batch['chunk'].to_list()
    # create embeddings
    res = client.embeddings.create(input=texts, model=embed_model_id)
    embeds = [record.embedding for record in res.data]
    # create metadata
    metadata = [{
        'chunk': x['chunk'],
        'source': x['source']
    } for _, x in batch.iterrows()]
    to_upsert = list(zip(ids_batch, embeds, metadata))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

100%|███████████████████████████████████████████| 49/49 [06:00<00:00,  7.36s/it]


In [85]:
async def retrieve(query: str) -> list:
    # create query embedding
    res = client.embeddings.create(input=[query], model=embed_model_id)
    xq = res.data[0].embedding
    # get relevant contexts from pinecone
    res = index.query(xq, top_k=5, include_metadata=True)
    # get list of retrieved texts
    contexts = [x['metadata']['chunk'] for x in res['matches']]
    return contexts

In [87]:
async def rag(query: str, contexts: list) -> str:
    print("> RAG Called")  # we'll add this so we can see when this is being used
    context_str = "\n".join(contexts)
    # place query and contexts into RAG prompt
    prompt = f"""You are a helpful assistant, below is a query from a user and
    some relevant contexts. Answer the question given the information in those
    contexts. If you cannot find the answer to the question, say "I don't know".

    Contexts:
    {context_str}

    Query: {query}

    Answer: """
    # generate answer
    res = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=0.0,
        max_tokens=100
    )
    return res.choices[0].text

In [92]:
yaml_content = """
models:
- type: main
  engine: openai
  model: text-davinci-003
"""

rag_colang_content = """
# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."
    "Sorry I can't talk about politics!"

define flow politics
    user ask politics
    bot answer politics
    bot offer help

# define RAG intents and flow
define user ask llama
    "tell me about llama 2?"
    "what is large language model"
    "where did meta's new model come from?"
    "how to llama?"
    "have you ever meta llama?"

define flow llama
    user ask llama
    $contexts = execute retrieve(query=$last_user_message)
    $answer = execute rag(query=$last_user_message, contexts=$contexts)
    bot $answer
"""

In [93]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
    colang_content=rag_colang_content,
    yaml_content=yaml_content
)
# create rails
rag_rails = LLMRails(config)

In [94]:
rag_rails.register_action(action=retrieve, name="retrieve")
rag_rails.register_action(action=rag, name="rag")

In [96]:
await rag_rails.generate_async(prompt="hello")

Error while execution generate_user_intent: 'NoneType' object has no attribute 'get'


"I'm sorry, an internal error has occurred."

In [97]:
await rag_rails.generate_async(prompt="tell me about llama 2")

Error while execution generate_user_intent: 'NoneType' object has no attribute 'get'


"I'm sorry, an internal error has occurred."